In [61]:
from datetime import datetime
import yfinance as yf
import pandas as pd
from tqdm import tqdm
import math
import statistics
from scipy.optimize import minimize, basinhopping
import plotly.express as px
from sympy import symbols, Eq, solve
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
# from google.colab import files
# To increase the width of the output
pd.set_option('display.width', 180)

# Parameters

There are the major parameters that are changeable in the code

In [62]:
start_date = datetime(2020,3,1)
end_date = datetime(2023,4,1)
no_of_years = 3
rf = 0.0739 #https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4 (10 year Gsec yield)
abt_rf = rf/4

market_ticker = "^CNXPSE"
tickers = ['BEL','BHEL','BPCL','COALINDIA','CONCOR','GAIL','HAL','HINDPETRO','IOC','IRCTC','LICI','NHPC','NMDC','NTPC','ONGC','OIL','PFC','POWERGRID','RECLTD','SAIL']

no_of_steps = 75

# Get Data

Get data of market and the shares you want in the portfolio

## Market Data

Here the market is NIFTY PSE + calculation of Rm + calulation of Returns in percent form

In [63]:
niftyPSE = yf.Ticker(market_ticker).history(start=start_date, end=end_date, interval='1mo')

temp = [0]
count = 0
for y in range(1,len(niftyPSE["Close"])): #start from 1 because 1st entry mai return nahi aayenge
  temp.append(((niftyPSE["Close"][y] + niftyPSE["Dividends"][y]) / niftyPSE["Close"][y-1]) - 1)

niftyPSE["Return"] = temp

# rm = ((niftyPSE["Close"][-1] / niftyPSE["Close"][1]) ** (1 / no_of_years)) - 1
rm = niftyPSE["Return"].iloc[1:].mean()

rm, niftyPSE

# whole python dictionary is printed here
# print(niftyPSE.info)

(0.02153183180145267,
                                   Open         High          Low        Close      Volume  Dividends  Stock Splits    Return
 Date                                                                                                                        
 2020-03-01 00:00:00+05:30  2768.949951  2814.050049  1952.199951  2214.149902    68423500        0.0           0.0  0.000000
 2020-04-01 00:00:00+05:30  2203.199951  2473.050049  2118.399902  2439.300049    40658500        0.0           0.0  0.101687
 2020-05-01 00:00:00+05:30  2368.949951  2404.149902  2153.750000  2369.000000    51591700        0.0           0.0 -0.028820
 2020-06-01 00:00:00+05:30  2396.000000  2639.550049  2378.199951  2493.949951    75063200        0.0           0.0  0.052744
 2020-07-01 00:00:00+05:30  2490.199951  2633.199951  2359.949951  2485.399902    66477400        0.0           0.0 -0.003428
 2020-08-01 00:00:00+05:30  2489.000000  2705.800049  2451.100098  2559.149902    64536000      

## Individual Shares Data

Get individual shares data which want to be included in the portfolio
"data" is a dict of dataframes, key name is the ticker name and the value contains the data for that share in the form of a df

In [64]:
data = {}

for x in tickers:
  if len(yf.Ticker(x + ".NS").history(start=start_date, end=end_date, interval='1mo')) == ((no_of_years*12) + 1):
    data[x] = yf.Ticker(x + ".NS").history(start=start_date, end=end_date, interval='1mo')
  else:
    print(x, len(yf.Ticker(x + ".NS").history(start=start_date, end=end_date, interval='1mo')))

data

LICI 10


{'BEL':                                  Open        High         Low       Close      Volume  Dividends  Stock Splits
 Date                                                                                                          
 2020-03-01 00:00:00+05:30   23.261140   24.531321   17.139788   22.786736   964029063   0.000000           0.0
 2020-04-01 00:00:00+05:30   22.587794   24.209952   19.925006   22.067478   795107448   0.000000           0.0
 2020-05-01 00:00:00+05:30   22.067479   22.067479   17.675408   21.179882   989872074   0.000000           0.0
 2020-06-01 00:00:00+05:30   21.424734   28.831572   20.628958   27.132896  1234457178   0.000000           0.0
 2020-07-01 00:00:00+05:30   27.132896   32.259527   26.796221   29.336584  1315473423   0.000000           0.0
 2020-08-01 00:00:00+05:30   29.382499   36.253720   29.091734   32.473782  1158783705   0.000000           0.0
 2020-09-01 00:00:00+05:30   32.749240   33.774569   27.438968   29.305981   626753472   0.466667

# Calculation of basic statistics of individual shares

Here, we are calculating returns (including dividends), variance, std dev, beta and expected return from CAPM and storing in the dict called "derived_data" where the key is the ticker name and the value is the dict containing key value pairs of the statistics mentioned above

In [107]:
statistics.variance(niftyPSE["Return"].iloc[1:])

0.0038186250114011734

In [65]:
# Calculating month on month return, x-xbar and (x-xbar)^2

# Calculation of monthly Returns
for x in data:
  temp = [0]
  count = 0
  for y in range(1,len(data[x]["Close"])): #start from 1 because 1st entry mai return nahi aayenge
    temp.append(((data[x]["Close"][y] + data[x]["Dividends"][y]) / data[x]["Close"][y-1]) - 1)

  data[x]["Return"] = temp

# Calculation of x-xbar
for x in data:
  temp = [0]
  count = 0
  mean = data[x]["Return"].iloc[1:].mean()
  for y in range(1,len(data[x]["Close"])): #start from 1 because 1st entry mai return nahi aayenge
    temp.append(data[x]["Return"][y] - mean)

  data[x]["XminusXbar"] = temp

# Calculation of (x-xbar)^2
for x in data:
  temp = [0]
  count = 0
  for y in range(1,len(data[x]["Close"])): #start from 1 because 1st entry mai return nahi aayenge
    temp.append(data[x]["XminusXbar"][y] * data[x]["XminusXbar"][y])

  data[x]["XminusXbar^2"] = temp

# Calculations wala DF
derived_data = {}

for x in data:
  cov_wrt_index = pd.DataFrame({x:data[x]["Return"].iloc[1:].tolist(), "Index":niftyPSE["Return"].iloc[1:].tolist()}).cov().iloc[0,1]
  var_index = pd.DataFrame({x:data[x]["Return"].iloc[1:].tolist(), "Index":niftyPSE["Return"].iloc[1:].tolist()}).cov().iloc[1,1]

  derived_data[x] = {
      "mean" : data[x]["Return"].iloc[1:].mean(),
      "sum_X_min_Xbar_square" : data[x]["XminusXbar^2"].sum(),
      "variance" : (data[x]["XminusXbar^2"].sum() / ((no_of_years*12)-1)),
      # "variance_temp" : statistics.variance(data[x]["Return"].iloc[1:]),
      "std_dev" : math.sqrt(data[x]["XminusXbar^2"].sum() / ((no_of_years*12)-1)),
      "cov_wrt_index":pd.DataFrame({x:data[x]["Return"].iloc[1:].tolist(), "Index":niftyPSE["Return"].iloc[1:].tolist()}).cov().iloc[0,1],
      "beta" : cov_wrt_index / var_index,

      "expected_return_capm" : (rf + ((cov_wrt_index / var_index) * (rm*12 - rf))),
      "expected_return_capm_monthly" : ((rf/12) + ((cov_wrt_index / var_index) * (rm - (rf/12)))),

      "beta_x_sd_market" : ((cov_wrt_index / var_index) * (statistics.stdev(niftyPSE["Return"].iloc[1:]))),
      "sys_risk" : (((cov_wrt_index / var_index) * (statistics.stdev(niftyPSE["Return"].iloc[1:])))**2), # (beta * std dev market) ^2
      "unsys_risk" : ((data[x]["XminusXbar^2"].sum() / ((no_of_years*12)-1)) - (((cov_wrt_index / var_index) * (statistics.stdev(niftyPSE["Return"].iloc[1:])))**2)), #variance - systematic risk

      "excess_mean_return_over_rm" : (data[x]["Return"].iloc[1:].mean() - (rm)),
      "excess_mean_return_over_rf" : (data[x]["Return"].iloc[1:].mean() - (rf/12)),
      "excess_return_to_beta" : ((data[x]["Return"].iloc[1:].mean() - (rf/12)) / (cov_wrt_index / var_index)),
      "ri_less_rf_x_beta_to_unsys" : ( ((data[x]["Return"].iloc[1:].mean() - (rf/12)) * (cov_wrt_index / var_index)) / ((data[x]["XminusXbar^2"].sum() / ((no_of_years*12)-1)) - (((cov_wrt_index / var_index) * (statistics.stdev(niftyPSE["Return"].iloc[1:])))**2))),
      "beta_square_to_unsys_square" : (((cov_wrt_index / var_index)**2) / (((data[x]["XminusXbar^2"].sum() / ((no_of_years*12)-1)) - (((cov_wrt_index / var_index) * (statistics.stdev(niftyPSE["Return"].iloc[1:])))**2))))
  }

derived_data

{'BEL': {'mean': 0.04656214547562422,
  'sum_X_min_Xbar_square': 0.28710578585515123,
  'variance': 0.00820302245300432,
  'std_dev': 0.09057053854871527,
  'cov_wrt_index': 0.0029396286002594772,
  'beta': 0.7698133730027696,
  'expected_return_capm': 0.2159166965271503,
  'expected_return_capm_monthly': 0.017993058043929193,
  'beta_x_sd_market': 0.04757063598630103,
  'sys_risk': 0.0022629654081411585,
  'unsys_risk': 0.005940057044863162,
  'excess_mean_return_over_rm': 0.02503031367417155,
  'excess_mean_return_over_rf': 0.040403812142290885,
  'excess_return_to_beta': 0.052485204283591366,
  'ri_less_rf_x_beta_to_unsys': 5.236211482905669,
  'beta_square_to_unsys_square': 99.76547780233533},
 'BHEL': {'mean': 0.04464589880656383,
  'sum_X_min_Xbar_square': 0.783531946462999,
  'variance': 0.02238662704179997,
  'std_dev': 0.14962161288329962,
  'cov_wrt_index': 0.006490763186970428,
  'beta': 1.6997644878958047,
  'expected_return_capm': 0.38747592100995765,
  'expected_return_ca

## Correlation data

Not really useful as covariance is used

In [66]:
corr_data = {}

for x in data:
  corr_data[x] = data[x]["Return"].iloc[1:].tolist()

corr_df = pd.DataFrame(corr_data)

corr_matrix = corr_df.corr()

corr_matrix

,BEL,BHEL,BPCL,COALINDIA,CONCOR,GAIL,HAL,HINDPETRO,IOC,IRCTC,NHPC,NMDC,NTPC,ONGC,OIL,PFC,POWERGRID,RECLTD,SAIL
BEL,1.000000,0.458244,0.279059,0.312393,0.274301,0.333944,0.584226,0.465179,0.153074,0.140456,0.430409,0.203770,0.265351,0.158324,0.144356,0.549721,0.400855,0.564447,0.306490
BHEL,0.458244,1.000000,0.569391,0.481138,0.593164,0.534157,0.432613,0.572155,0.616320,0.375250,0.493873,0.407236,0.545967,0.400428,0.171060,0.442128,0.404152,0.514465,0.334476
BPCL,0.279059,0.569391,1.000000,0.403083,0.608999,0.651854,0.443942,0.837745,0.724892,0.461787,0.365414,0.426067,0.410895,0.531109,0.303246,0.530258,0.428799,0.467097,0.512926
COALINDIA,0.312393,0.481138,0.403083,1.000000,0.417610,0.665949,0.272042,0.469711,0.523859,0.508989,0.446194,0.362357,0.730397,0.700060,0.543182,0.574768,0.541235,0.504512,0.450468
CONCOR,0.274301,0.593164,0.608999,0.417610,1.000000,0.533593,0.422315,0.506474,0.507765,0.367043,0.289421,0.220514,0.347755,0.246410,0.169171,0.293045,0.484161,0.236575,0.207490
GAIL,0.333944,0.534157,0.651854,0.665949,0.533593,1.000000,0.424212,0.618748,0.627021,0.386612,0.290459,0.437859,0.529056,0.672426,0.445974,0.592195,0.458211,0.602185,0.532106
HAL,0.584226,0.432613,0.443942,0.272042,0.422315,0.424212,1.000000,0.422930,0.287967,0.176324,0.271207,0.062433,0.215798,0.242920,0.200874,0.352183,0.413438,0.418035,0.079483
HINDPETRO,0.465179,0.572155,0.837745,0.469711,0.506474,0.618748,0.422930,1.000000,0.773114,0.520274,0.430626,0.332860,0.451147,0.511154,0.410745,0.633073,0.464582,0.659642,0.414668
IOC,0.153074,0.616320,0.724892,0.523859,0.507765,0.627021,0.287967,0.773114,1.000000,0.420088,0.414386,0.465082,0.545608,0.605026,0.511830,0.459274,0.512136,0.463614,0.324690
IRCTC,0.140456,0.375250,0.461787,0.508989,0.367043,0.386612,0.176324,0.520274,0.420088,1.000000,0.354185,0.036975,0.581791,0.528061,0.424816,0.200540,0.317798,0.132988,0.308929


## Covariance Data

Extremely important as this is will be used for calculating the std dev for the Markowitz Model

In [67]:
cov_data = {}

for x in data:
  cov_data[x] = data[x]["Return"].iloc[1:].tolist()

cov_df = pd.DataFrame(cov_data)

cov_matrix = cov_df.cov()

cov_matrix

,BEL,BHEL,BPCL,COALINDIA,CONCOR,GAIL,HAL,HINDPETRO,IOC,IRCTC,NHPC,NMDC,NTPC,ONGC,OIL,PFC,POWERGRID,RECLTD,SAIL
BEL,0.008203,0.006210,0.002096,0.002873,0.002302,0.002835,0.005631,0.003586,0.000997,0.001442,0.002696,0.002127,0.002067,0.001478,0.001551,0.004592,0.001987,0.004425,0.005024
BHEL,0.006210,0.022387,0.007064,0.007310,0.008225,0.007492,0.006888,0.007287,0.006633,0.006366,0.005110,0.007023,0.007024,0.006176,0.003036,0.006101,0.003309,0.006662,0.009058
BPCL,0.002096,0.007064,0.006875,0.003394,0.004680,0.005067,0.003917,0.005913,0.004323,0.004342,0.002095,0.004072,0.002930,0.004539,0.002983,0.004055,0.001945,0.003352,0.007698
COALINDIA,0.002873,0.007310,0.003394,0.010310,0.003930,0.006339,0.002939,0.004060,0.003826,0.005860,0.003133,0.004241,0.006377,0.007327,0.006543,0.005383,0.003007,0.004434,0.008278
CONCOR,0.002302,0.008225,0.004680,0.003930,0.008589,0.004636,0.004165,0.003995,0.003385,0.003857,0.001855,0.002355,0.002771,0.002354,0.001860,0.002505,0.002455,0.001898,0.003480
GAIL,0.002835,0.007492,0.005067,0.006339,0.004636,0.008787,0.004232,0.004937,0.004228,0.004109,0.001883,0.004731,0.004264,0.006498,0.004959,0.005120,0.002350,0.004886,0.009028
HAL,0.005631,0.006888,0.003917,0.002939,0.004165,0.004232,0.011324,0.003831,0.002204,0.002128,0.001996,0.000766,0.001975,0.002665,0.002536,0.003457,0.002407,0.003850,0.001531
HINDPETRO,0.003586,0.007287,0.005913,0.004060,0.003995,0.004937,0.003831,0.007245,0.004733,0.005021,0.002535,0.003266,0.003302,0.004485,0.004148,0.004970,0.002164,0.004860,0.006388
IOC,0.000997,0.006633,0.004323,0.003826,0.003385,0.004228,0.002204,0.004733,0.005173,0.003426,0.002061,0.003855,0.003374,0.004486,0.004367,0.003047,0.002016,0.002886,0.004227
IRCTC,0.001442,0.006366,0.004342,0.005860,0.003857,0.004109,0.002128,0.005021,0.003426,0.012857,0.002777,0.000483,0.005672,0.006172,0.005714,0.002097,0.001972,0.001305,0.006340


# Markowitz Model

## General code for efficiency frontier

This is the general code for the efficiency frontier found online where you generate random portfolios according to weights and find the return and risk for that. It works for less number of securities like 3-4 max but for more it starts failing as the feasible set keeps increasing so you have to keep generating more random portfolios which is time consuming and inefficient.

In [68]:
p_ret = [] # Define an empty array for portfolio returns
p_var = [] # Define an empty array for portfolio volatility
p_weights = [] # Define an empty array for asset weights
p_beta = []
p_sharpe = []

num_assets = len(data)
num_portfolios = 1000

exp_ret_arr = []
beta_arr = []
for x in derived_data:
  exp_ret_arr.append(derived_data[x]['mean'])
  beta_arr.append(derived_data[x]['beta'])

for portfolio in range(num_portfolios):
  weights = np.random.random(num_assets)
  weights = weights/np.sum(weights)
  while weights.min() < 0.01:
    weights = np.random.random(num_assets)
    weights = weights/np.sum(weights)
  p_weights.append(weights)
  returns = np.dot(weights, exp_ret_arr) # Returns are the product of individual expected returns of asset and its weights
  p_ret.append(returns)

  var = 0
  for x in range(len(weights)):
    for y in range(len(weights)):
      var += weights[x] * weights[y] * cov_matrix.iloc[x,y]

  sd = np.sqrt(var) # Daily standard deviation
  p_var.append(sd)
  betas = np.dot(weights, beta_arr)
  sharpe_ratio = (returns - (rf/12)) / sd
  p_beta.append(betas)
  p_sharpe.append(sharpe_ratio)

p_treynor = []
for x in range(len(p_ret)):
  p_treynor.append((p_ret[x] - (rf/12)) / p_beta[x])

portfolios = {'Returns':p_ret, 'Volatility':p_var, 'Beta': p_beta, 'Sharpe_Ratio':p_sharpe, 'Treynor_Ratio':p_treynor}

for x in range(len(data)):
    #print(counter, symbol)
    temp = []
    for y in p_weights:
      temp.append(y[x])
    portfolios[list(data.keys())[x]] = temp

portfolios_df = pd.DataFrame(portfolios)

fig = px.scatter(x=portfolios_df['Volatility'].to_list(), y=portfolios_df['Returns'].to_list(), title='Markowitz Feasible Set Random Portfolios')
fig.show()

## Specialised code for the Efficiency Frontier

This is the code where I used linear programming to find the minimum variance portfolios and the portfolios on the efficiency frontier. Constrained Linear Programming was used with several objective functions each serving a different purpose.
Maximum variance portfolio is also found along with the worst portfolios.
Dominant portfolio was found using the maximising the Sharpe Ratio and the CML was draw with than point.
Equal weightage portfolio is also found but it might be hidden in the graph as it lies in the feasible set. to see it toggle off the feasible set by clicking on the legend name of feasible set.

In [69]:
def objective_min_var(x):
  var = 0
  for z in range(len(x)):
    for y in range(len(x)):
      var += x[z] * x[y] * cov_matrix.iloc[z,y]
  return np.sqrt(var)

def objective_max_var(x):
  var = 0
  for z in range(len(x)):
    for y in range(len(x)):
      var += x[z] * x[y] * cov_matrix.iloc[z,y]
  return -np.sqrt(var)

# maximisation function for efficient frontier isliye - se multiply kiya hai kyuki scipy.optimise mai maximisation nahi hota only minisation hi hota hai
def objective_efficiency_frontier(x):
  return -np.dot(x, exp_ret_arr)

def objective_worst_portfolios(x):
  return np.dot(x, exp_ret_arr)

# maximisation function for efficient frontier isliye - se multiply kiya hai kyuki scipy.optimise mai maximisation nahi hota only minisation hi hota hai
def objective_max_sharpe(x):
  rp = np.dot(x, exp_ret_arr)
  rf_month = rf / 12
  var = 0
  for z in range(len(x)):
    for y in range(len(x)):
      var += x[z] * x[y] * cov_matrix.iloc[z,y]
  sd = np.sqrt(var)
  return -((rp - rf_month) / sd)

def constraint1(x):
  return np.sum(x) - 1

con1 = {'type': 'eq', 'fun': constraint1}

def constraint2(x):
  var = 0
  for z in range(len(x)):
    for y in range(len(x)):
      var += x[z] * x[y] * cov_matrix.iloc[z,y]
  sd = np.sqrt(var)
  return sd - target_sd

con2 = {'type': 'eq', 'fun': constraint2}

min = 0.0001
max = 1
b = (min, max)
bounds = ()
for i in range(num_assets):
  bounds += (b,)

def generate_random_guess():
  x = np.random.random(num_assets)
  x = x/np.sum(x)
  while x.min() < min:
    x = np.random.random(num_assets)
    x = x/np.sum(x)
  return x
x = np.random.random(num_assets)
x = x/np.sum(x)

def calc_sd(x):
  var = 0
  for z in range(len(x['x'])):
    for y in range(len(x['x'])):
      var += x['x'][z] * x['x'][y] * cov_matrix.iloc[z,y]
  sd = np.sqrt(var)
  return sd

def generate_portfolio_df(p_ret, p_var, p_beta, p_sharpe, p_weights):
  p_treynor = []
  for x in range(len(p_ret)):
    p_treynor.append((p_ret[x] - (rf/12)) / p_beta[x])
  portfolio = {'Returns':p_ret, 'Volatility':p_var, 'Beta' : p_beta, 'Sharpe_Ratio' : p_sharpe, 'Treynor_Ratio':p_treynor}
  for x in range(len(data)):
      temp = []
      for y in p_weights:
        temp.append(y[x])
      portfolio[list(data.keys())[x]] = temp
  return pd.DataFrame(portfolio)

method = 'SLSQP'
niter = 15
# method = 'COBYLA'
# method = 'BFGS'

# mvp = minimize (objective_min_var,x,method=method,bounds=bounds,constraints=[con1])
mvp = basinhopping (objective_min_var,x,minimizer_kwargs=dict(method=method,bounds=bounds,constraints=[con1]), niter=niter)
mvp_sd = calc_sd(mvp)
mvp_ret = np.dot(mvp['x'], exp_ret_arr)
mvp_beta = np.dot(mvp['x'], beta_arr)
mvp_sharpe = (mvp_ret - (rf/12)) / mvp_sd

# maxvp = minimize (objective_max_var,x,method=method,bounds=bounds,constraints=[con1])
maxvp = basinhopping (objective_max_var,x,minimizer_kwargs=dict(method=method,bounds=bounds,constraints=[con1]), niter=niter)
maxvp_sd = calc_sd(maxvp)
maxvp_ret = np.dot(maxvp['x'], exp_ret_arr)
maxvp_beta = np.dot(maxvp['x'], beta_arr)
maxvp_sharpe = (maxvp_ret - (rf/12)) / maxvp_sd

diff = maxvp_sd - mvp_sd
step = diff / no_of_steps

p_ret_best = [] # Define an empty array for portfolio returns
p_var_best = [] # Define an empty array for portfolio volatility
p_weights_best = [] # Define an empty array for asset weights
p_beta_best = []
p_sharpe_best = []
p_ret_ef = [] # Define an empty array for portfolio returns
p_var_ef = [] # Define an empty array for portfolio volatility
p_weights_ef = [] # Define an empty array for asset weights
p_beta_ef = []
p_sharpe_ef = []
p_ret_wp = [] # Define an empty array for portfolio returns
p_var_wp = [] # Define an empty array for portfolio volatility
p_weights_wp = [] # Define an empty array for asset weights
p_beta_wp = []
p_sharpe_wp = []

p_ret_best.append(mvp_ret)
p_var_best.append(mvp_sd)
p_weights_best.append(mvp['x'])
p_beta_best.append(mvp_beta)
p_sharpe_best.append(mvp_sharpe)

target_sd = mvp_sd
count_ef = 0
count_wp = 0
ef_x = mvp['x']
wp_x = mvp['x']
for i in tqdm(range(no_of_steps-1)):
  target_sd += step
  # temp = minimize(objective_efficiency_frontier,ef_x,method=method,bounds=bounds,constraints=[con1, con2])
  temp = basinhopping(objective_efficiency_frontier,ef_x,minimizer_kwargs=dict(method=method,bounds=bounds,constraints=[con1, con2]), niter=niter)
  while temp['success'] != True:
    print("error while optimising count ef = ", count_ef)
    print(temp)
    # temp = minimize(objective_efficiency_frontier,generate_random_guess(),method=method,bounds=bounds,constraints=[con1, con2])
    temp = basinhopping(objective_efficiency_frontier,ef_x,minimizer_kwargs=dict(method=method,bounds=bounds,constraints=[con1, con2]), niter=niter)
    count_ef += 1

  p_ret_ef.append(np.dot(temp['x'], exp_ret_arr))
  p_var_ef.append(target_sd)
  p_weights_ef.append(temp['x'])
  p_beta_ef.append(np.dot(temp['x'], beta_arr))
  p_sharpe_ef.append((np.dot(temp['x'], exp_ret_arr) - (rf/12)) / target_sd)
  ef_x = temp['x']

  # temp1 = minimize(objective_worst_portfolios,wp_x,method=method,bounds=bounds,constraints=[con1, con2])
  temp1 = basinhopping(objective_worst_portfolios,wp_x,minimizer_kwargs=dict(method=method,bounds=bounds,constraints=[con1, con2]), niter=niter)
  while temp1['success'] != True:
    print("error while optimising count wp = ", count_wp)
    print(temp1)
    temp1 = basinhopping(objective_worst_portfolios,wp_x,minimizer_kwargs=dict(method=method,bounds=bounds,constraints=[con1, con2]), niter=niter)
    # temp1 = minimize(objective_worst_portfolios,generate_random_guess(),method=method,bounds=bounds,constraints=[con1, con2])
    count_wp += 1
  p_ret_wp.append(np.dot(temp1['x'], exp_ret_arr))
  p_var_wp.append(target_sd)
  p_weights_wp.append(temp1['x'])
  p_beta_wp.append(np.dot(temp['x'], beta_arr))
  p_sharpe_wp.append((np.dot(temp['x'], exp_ret_arr) - (rf/12)) / target_sd)
  wp_x = temp['x']

  if temp['success'] != True:
    print("error while optimising")
    break
  if temp1['success'] != True:
    print("error while optimising")
    break

p_ret_best.append(maxvp_ret)
p_var_best.append(maxvp_sd)
p_weights_best.append(maxvp['x'])
p_beta_best.append(maxvp_beta)
p_sharpe_best.append(maxvp_sharpe)

# max_sharpe_port = minimize(objective_max_sharpe,generate_random_guess(),method=method,bounds=bounds,constraints=[con1])
max_sharpe_port = basinhopping(objective_max_sharpe,generate_random_guess(),minimizer_kwargs=dict(method=method,bounds=bounds,constraints=[con1]), niter=niter)
max_sharpe_port_sd = calc_sd(max_sharpe_port)
max_sharpe_port_ret = np.dot(max_sharpe_port['x'], exp_ret_arr)
max_sharpe_port_beta = np.dot(max_sharpe_port['x'], beta_arr)
max_sharpe_port_sharpe = (max_sharpe_port_ret - (rf/12)) / max_sharpe_port_sd

p_ret_best.append(max_sharpe_port_ret)
p_var_best.append(max_sharpe_port_sd)
p_weights_best.append(max_sharpe_port['x'])
p_beta_best.append(max_sharpe_port_beta)
p_sharpe_best.append(max_sharpe_port_sharpe)

equal_weights = np.array([1/num_assets] * num_assets)
equal_weights_ret = np.dot(equal_weights, exp_ret_arr)
equal_weights_var = 0
for z in range(len(equal_weights)):
  for y in range(len(equal_weights)):
    equal_weights_var += equal_weights[z] * equal_weights[y] * cov_matrix.iloc[z,y]
equal_weights_sd = np.sqrt(equal_weights_var)
equal_weights_beta = np.dot(equal_weights, beta_arr)
equal_weights_sharpe = (equal_weights_ret - (rf/12)) / equal_weights_sd

p_ret_best.append(equal_weights_ret)
p_var_best.append(equal_weights_sd)
p_weights_best.append(equal_weights)
p_beta_best.append(equal_weights_beta)
p_sharpe_best.append(equal_weights_sharpe)

portfolios_best_df = generate_portfolio_df(p_ret_best, p_var_best, p_beta_best, p_sharpe_best, p_weights_best)

portfolios_ef_df = generate_portfolio_df(p_ret_ef, p_var_ef, p_beta_ef, p_sharpe_ef, p_weights_ef)

portfolios_wp_df = generate_portfolio_df(p_ret_wp, p_var_wp, p_beta_wp, p_sharpe_wp, p_weights_wp)

# fig1 = px.scatter(x=portfolios_ef_df['Volatility'].to_list(), y=portfolios_ef_df['Returns'].to_list())
# fig1.show()

plot_best_df = portfolios_best_df.copy(deep=True)
plot_best_df["Classifier"]=["best"]*(len(portfolios_best_df))

plot_ef_df = portfolios_ef_df.copy(deep=True)
plot_ef_df["Classifier"]=["efficient_frontier"]*(len(portfolios_ef_df))

plot_df = portfolios_df.copy(deep=True)
plot_df["Classifier"]=["feasible_set"]*(len(portfolios_df))

plot_wp_df = portfolios_wp_df.copy(deep=True)
plot_wp_df["Classifier"]=["worst_portfolios"]*(len(portfolios_wp_df))

plot_final = pd.concat([plot_best_df, plot_ef_df,plot_df,plot_wp_df])

fig2 = px.scatter(plot_final,x=plot_final['Volatility'], y=plot_final['Returns'], color="Classifier", color_discrete_map={"efficient_frontier": 'blue', "feasible_set": 'black', "worst_portfolios": 'red', "best":"green"}, hover_data=plot_final, title="Markowitz Efficiency Frontier")
# fig2.show()

slope = (np.dot(max_sharpe_port['x'], exp_ret_arr) - (rf/12)) / (max_sharpe_port_sd - 0)
y = (rf/12) + (slope * 0.1)

temp = {"Volatility":[0,0.1], "Returns":[(rf/12), y]}
temp_df = pd.DataFrame(temp)

fig2.add_trace(px.line(temp_df, x=temp_df['Volatility'], y=temp_df['Returns']).data[0])
fig2.show()

# Superimpose graph ka code
# fig2.add_trace(px.line(temp_df, x=temp_df['Volatility'], y=temp_df['Returns']).data[0])

100%|██████████| 74/74 [1:14:05<00:00, 60.07s/it] 


# Sharpe Optimal Portfolio Model

In [75]:
sharpe_df = pd.DataFrame(derived_data).transpose().sort_values('excess_return_to_beta', ascending=False)

ri_less_rf_x_beta_to_unsys = []
beta_square_to_unsys_square = []
sum_ri_less_rf_x_beta_to_unsys = []
sum_beta_square_to_unsys_square = []
c = []
market_var = statistics.variance(niftyPSE["Return"].iloc[1:])

for x in range(len(sharpe_df)):
    temp = sharpe_df['ri_less_rf_x_beta_to_unsys'][x]
    temp1 = sharpe_df['beta_square_to_unsys_square'][x]

    ri_less_rf_x_beta_to_unsys.append(temp)
    beta_square_to_unsys_square.append(temp1)

    temp2 = sum(ri_less_rf_x_beta_to_unsys)
    temp3 = sum(beta_square_to_unsys_square)

    sum_ri_less_rf_x_beta_to_unsys.append(temp2)
    sum_beta_square_to_unsys_square.append(temp3)

    temp4 = (market_var*temp2) / (1 + (market_var*temp3))

    c.append(temp4)

sharpe_df['ri_less_rf_x_beta_to_unsys'] = ri_less_rf_x_beta_to_unsys
sharpe_df['beta_square_to_unsys_square'] = beta_square_to_unsys_square
sharpe_df['sum_ri_less_rf_x_beta_to_unsys'] = sum_ri_less_rf_x_beta_to_unsys
sharpe_df['sum_beta_square_to_unsys_square'] = sum_beta_square_to_unsys_square
sharpe_df['c'] = c

sharpe_df_original = sharpe_df.copy(deep=True)

sharpe_df

,mean,sum_X_min_Xbar_square,variance,std_dev,cov_wrt_index,beta,expected_return_capm,expected_return_capm_monthly,beta_x_sd_market,sys_risk,unsys_risk,excess_mean_return_over_rm,excess_mean_return_over_rf,excess_return_to_beta,ri_less_rf_x_beta_to_unsys,beta_square_to_unsys_square,sum_ri_less_rf_x_beta_to_unsys,sum_beta_square_to_unsys_square,c
HAL,0.058588,0.396330,0.011324,0.106413,0.003186,0.834416,0.227835,0.018986,0.051563,0.002659,0.008665,0.037056,0.052430,0.062834,5.048844,80.351929,5.048844,80.351929,0.014753
BEL,0.046562,0.287106,0.008203,0.090571,0.002940,0.769813,0.215917,0.017993,0.047571,0.002263,0.005940,0.025030,0.040404,0.052485,5.236211,99.765478,10.285055,180.117407,0.023270
NHPC,0.032328,0.167354,0.004782,0.069149,0.002323,0.608280,0.186117,0.015510,0.037589,0.001413,0.003369,0.010796,0.026169,0.043022,4.725414,109.838191,15.010470,289.955598,0.027201
POWERGRID,0.031482,0.104790,0.002994,0.054717,0.002455,0.642886,0.192501,0.016042,0.039727,0.001578,0.001416,0.009951,0.025324,0.039391,11.499600,291.932647,26.510070,581.888245,0.031419
OIL,0.049304,0.492542,0.014073,0.118628,0.004190,1.097161,0.276306,0.023026,0.067799,0.004597,0.009476,0.027773,0.043146,0.039325,4.995637,127.034009,31.505707,708.922254,0.032453
NMDC,0.038297,0.464934,0.013284,0.115256,0.003496,0.915542,0.242801,0.020233,0.056576,0.003201,0.010083,0.016765,0.032139,0.035103,2.918203,83.131721,34.423910,792.053975,0.032662
RECLTD,0.036997,0.262198,0.007491,0.086553,0.003553,0.930532,0.245566,0.020464,0.057502,0.003307,0.004185,0.015466,0.030839,0.033141,6.857289,206.910421,41.281199,998.964396,0.032741
SAIL,0.057525,1.146535,0.032758,0.180992,0.006438,1.685936,0.384925,0.032077,0.104182,0.010854,0.021904,0.035993,0.051367,0.030468,3.953619,129.764442,45.234818,1128.728838,0.032529
IRCTC,0.036565,0.449986,0.012857,0.113388,0.003957,1.036150,0.265051,0.022088,0.064029,0.004100,0.008757,0.015033,0.030407,0.029346,3.597782,122.599513,48.832600,1251.328351,0.032271
ONGC,0.039971,0.371899,0.010626,0.103081,0.004965,1.300136,0.313752,0.026146,0.080342,0.006455,0.004171,0.018439,0.033812,0.026007,10.539882,405.275695,59.372481,1656.604046,0.030948


In [76]:
temp = 0
count = 0
for x in range(len(sharpe_df)):
    if temp < sharpe_df['c'][x]:
        temp = sharpe_df['c'][x]
    else:
        count = x
        break

sharpe_df = sharpe_df[:][:count]

z = []
for x in range(len(sharpe_df)):
    temp = (sharpe_df['beta'][x] / sharpe_df['unsys_risk'][x]) * (sharpe_df['excess_return_to_beta'][x] - sharpe_df['c'][x])
    z.append(temp)

sharpe_df['z'] = z

weights = z / sum(z)
sharpe_df['final_weights'] = weights

sharpe_df

,mean,sum_X_min_Xbar_square,variance,std_dev,cov_wrt_index,beta,expected_return_capm,expected_return_capm_monthly,beta_x_sd_market,sys_risk,...,excess_mean_return_over_rm,excess_mean_return_over_rf,excess_return_to_beta,ri_less_rf_x_beta_to_unsys,beta_square_to_unsys_square,sum_ri_less_rf_x_beta_to_unsys,sum_beta_square_to_unsys_square,c,z,final_weights
HAL,0.058588,0.396330,0.011324,0.106413,0.003186,0.834416,0.227835,0.018986,0.051563,0.002659,...,0.037056,0.052430,0.062834,5.048844,80.351929,5.048844,80.351929,0.014753,4.630085,0.289389
BEL,0.046562,0.287106,0.008203,0.090571,0.002940,0.769813,0.215917,0.017993,0.047571,0.002263,...,0.025030,0.040404,0.052485,5.236211,99.765478,10.285055,180.117407,0.023270,3.786229,0.236646
NHPC,0.032328,0.167354,0.004782,0.069149,0.002323,0.608280,0.186117,0.015510,0.037589,0.001413,...,0.010796,0.026169,0.043022,4.725414,109.838191,15.010470,289.955598,0.027201,2.856707,0.178549
POWERGRID,0.031482,0.104790,0.002994,0.054717,0.002455,0.642886,0.192501,0.016042,0.039727,0.001578,...,0.009951,0.025324,0.039391,11.499600,291.932647,26.510070,581.888245,0.031419,3.620247,0.226272
OIL,0.049304,0.492542,0.014073,0.118628,0.004190,1.097161,0.276306,0.023026,0.067799,0.004597,...,0.027773,0.043146,0.039325,4.995637,127.034009,31.505707,708.922254,0.032453,0.795640,0.049729
NMDC,0.038297,0.464934,0.013284,0.115256,0.003496,0.915542,0.242801,0.020233,0.056576,0.003201,...,0.016765,0.032139,0.035103,2.918203,83.131721,34.423910,792.053975,0.032662,0.221634,0.013853
RECLTD,0.036997,0.262198,0.007491,0.086553,0.003553,0.930532,0.245566,0.020464,0.057502,0.003307,...,0.015466,0.030839,0.033141,6.857289,206.910421,41.281199,998.964396,0.032741,0.089005,0.005563


In [77]:
sharpe_data = {}
for x in range(len(sharpe_df)):
    sharpe_data[sharpe_df.index.tolist()[x]] = data[sharpe_df.index.tolist()[x]]["Return"].iloc[1:].tolist()

sharpe_cov_df = pd.DataFrame(sharpe_data)

sharpe_cov_matrix = sharpe_cov_df.cov()

sharpe_cov_matrix

,HAL,BEL,NHPC,POWERGRID,OIL,NMDC,RECLTD
HAL,0.011324,0.005631,0.001996,0.002407,0.002536,0.000766,0.003850
BEL,0.005631,0.008203,0.002696,0.001987,0.001551,0.002127,0.004425
NHPC,0.001996,0.002696,0.004782,0.001872,0.001419,0.001347,0.002552
POWERGRID,0.002407,0.001987,0.001872,0.002994,0.002968,0.001830,0.001976
OIL,0.002536,0.001551,0.001419,0.002968,0.014073,0.000512,0.003505
NMDC,0.000766,0.002127,0.001347,0.001830,0.000512,0.013284,0.003924
RECLTD,0.003850,0.004425,0.002552,0.001976,0.003505,0.003924,0.007491


In [92]:
# sharpe_return = np.dot(sharpe_df['mean'].tolist(), sharpe_df['final_weights'].tolist())

# sharpe_var = 0
# for z in range(len(sharpe_df)):
#   for y in range(len(sharpe_df)):
#       sharpe_var += sharpe_df['final_weights'][z] * sharpe_df['final_weights'][y] * sharpe_cov_matrix.iloc[z,y]
# sharpe_sd = np.sqrt(sharpe_var)

alpha_val = []
for x in range(len(sharpe_df)):
  alpha_val.append(sharpe_df['mean'] - (sharpe_df['beta'] * rm))

# weighted_alpha = (np.dot(sharpe_df['excess_mean_return_over_rm'].tolist(), sharpe_df['final_weights'].tolist()))
weighted_alpha = (np.dot(alpha_val, sharpe_df['final_weights'].tolist())[0])
weighted_beta = ((np.dot(sharpe_df['beta'].tolist(), sharpe_df['final_weights'].tolist())))
sharpe_return = weighted_alpha + weighted_beta * rm

var_market = statistics.variance(niftyPSE["Return"].iloc[1:].tolist())
weighted_unsys_risk = ((np.dot((sharpe_df['final_weights'] * sharpe_df['final_weights']), sharpe_df['unsys_risk'].tolist())))

sharpe_variance = (weighted_beta ** 2) * var_market + weighted_unsys_risk
sharpe_sd = np.sqrt(sharpe_variance)
sharpe_ratio_of_sharpe_port = (sharpe_return - (rf/12)) / sharpe_sd
treynor_ratio_of_sharpe_port = (sharpe_return - (rf/12)) / weighted_beta

sharpe_portfolio = {'Returns':[sharpe_return], 'Volatility':[sharpe_sd], 'Alpha': [weighted_alpha], 'Beta':[weighted_beta], 'Sharpe_Ratio':[sharpe_ratio_of_sharpe_port], 'Treynor_Ratio':treynor_ratio_of_sharpe_port}
for x in range(len(data)):
    temp = []
    if list(data.keys())[x] in sharpe_df.index.tolist():
      temp.append(sharpe_df['final_weights'][list(data.keys())[x]])
    else:
      temp.append(0)
    sharpe_portfolio[list(data.keys())[x]] = temp
sharpe_portfolio_df = pd.DataFrame(sharpe_portfolio)

sharpe_portfolio_df

,Returns,Volatility,Alpha,Beta,Sharpe_Ratio,Treynor_Ratio,BEL,BHEL,BPCL,COALINDIA,...,IRCTC,NHPC,NMDC,NTPC,ONGC,OIL,PFC,POWERGRID,RECLTD,SAIL
0,0.044057,0.058416,0.027905,0.750138,0.648775,0.050523,0.236646,0,0,0,...,0,0.178549,0.013853,0,0,0.049729,0,0.226272,0.005563,0


# Capital Asset Pricing Model

In [93]:
capm_data = {}
for x in derived_data:
    if derived_data[x]['mean'] > derived_data[x]['expected_return_capm_monthly']:
        capm_data[x] = data[x]

capm_exp_ret_arr = []
capm_beta_arr = []
for x in derived_data:
    if x in list(capm_data.keys()):
        capm_exp_ret_arr.append(derived_data[x]['mean'])
        capm_beta_arr.append(derived_data[x]['beta'])

capm_num_assets = len(list(capm_data.keys()))

capm_cov_data = {}

for x in capm_data:
  capm_cov_data[x] = capm_data[x]["Return"].iloc[1:].tolist()

capm_cov_df = pd.DataFrame(capm_cov_data)

capm_cov_matrix = capm_cov_df.cov()

print("No of underpriced securities:", capm_num_assets)
print("Total securities: ", len(data))
capm_cov_matrix

No of underpriced securities: 17
Total securities:  19


,BEL,BHEL,COALINDIA,CONCOR,GAIL,HAL,IOC,IRCTC,NHPC,NMDC,NTPC,ONGC,OIL,PFC,POWERGRID,RECLTD,SAIL
BEL,0.008203,0.006210,0.002873,0.002302,0.002835,0.005631,0.000997,0.001442,0.002696,0.002127,0.002067,0.001478,0.001551,0.004592,0.001987,0.004425,0.005024
BHEL,0.006210,0.022387,0.007310,0.008225,0.007492,0.006888,0.006633,0.006366,0.005110,0.007023,0.007024,0.006176,0.003036,0.006101,0.003309,0.006662,0.009058
COALINDIA,0.002873,0.007310,0.010310,0.003930,0.006339,0.002939,0.003826,0.005860,0.003133,0.004241,0.006377,0.007327,0.006543,0.005383,0.003007,0.004434,0.008278
CONCOR,0.002302,0.008225,0.003930,0.008589,0.004636,0.004165,0.003385,0.003857,0.001855,0.002355,0.002771,0.002354,0.001860,0.002505,0.002455,0.001898,0.003480
GAIL,0.002835,0.007492,0.006339,0.004636,0.008787,0.004232,0.004228,0.004109,0.001883,0.004731,0.004264,0.006498,0.004959,0.005120,0.002350,0.004886,0.009028
HAL,0.005631,0.006888,0.002939,0.004165,0.004232,0.011324,0.002204,0.002128,0.001996,0.000766,0.001975,0.002665,0.002536,0.003457,0.002407,0.003850,0.001531
IOC,0.000997,0.006633,0.003826,0.003385,0.004228,0.002204,0.005173,0.003426,0.002061,0.003855,0.003374,0.004486,0.004367,0.003047,0.002016,0.002886,0.004227
IRCTC,0.001442,0.006366,0.005860,0.003857,0.004109,0.002128,0.003426,0.012857,0.002777,0.000483,0.005672,0.006172,0.005714,0.002097,0.001972,0.001305,0.006340
NHPC,0.002696,0.005110,0.003133,0.001855,0.001883,0.001996,0.002061,0.002777,0.004782,0.001347,0.003281,0.001000,0.001419,0.002877,0.001872,0.002552,0.002976
NMDC,0.002127,0.007023,0.004241,0.002355,0.004731,0.000766,0.003855,0.000483,0.001347,0.013284,0.003152,0.004904,0.000512,0.004783,0.001830,0.003924,0.014217


In [99]:
def capm_objective_min_var(x):
  var = 0
  for z in range(len(x)):
    for y in range(len(x)):
      var += x[z] * x[y] * capm_cov_matrix.iloc[z,y]
  return np.sqrt(var)

def capm_objective_max_var(x):
  var = 0
  for z in range(len(x)):
    for y in range(len(x)):
      var += x[z] * x[y] * capm_cov_matrix.iloc[z,y]
  return -np.sqrt(var)

# maximisation function for efficient frontier isliye - se multiply kiya hai kyuki scipy.optimise mai maximisation nahi hota only minisation hi hota hai
def capm_objective_efficiency_frontier(x):
  return -np.dot(x, capm_exp_ret_arr)

def capm_objective_worst_portfolios(x):
  return np.dot(x, capm_exp_ret_arr)

# maximisation function for efficient frontier isliye - se multiply kiya hai kyuki scipy.optimise mai maximisation nahi hota only minisation hi hota hai
def capm_objective_max_sharpe(x):
  rp = np.dot(x, capm_exp_ret_arr)
  rf_month = rf / 12
  var = 0
  for z in range(len(x)):
    for y in range(len(x)):
      var += x[z] * x[y] * capm_cov_matrix.iloc[z,y]
  sd = np.sqrt(var)
  return -((rp - rf_month) / sd)

def capm_constraint1(x):
  return np.sum(x) - 1

capm_con1 = {'type': 'eq', 'fun': capm_constraint1}

def capm_constraint2(x):
  var = 0
  for z in range(len(x)):
    for y in range(len(x)):
      var += x[z] * x[y] * capm_cov_matrix.iloc[z,y]
  sd = np.sqrt(var)
  return sd - capm_target_sd

capm_con2 = {'type': 'eq', 'fun': capm_constraint2}

def capm_calc_sd(x):
  var = 0
  for z in range(len(x['x'])):
    for y in range(len(x['x'])):
      var += x['x'][z] * x['x'][y] * capm_cov_matrix.iloc[z,y]
  sd = np.sqrt(var)
  return sd

def capm_generate_portfolio_df(p_ret, p_var, p_beta, p_sharpe, p_weights):
  p_treynor = []
  for x in range(len(p_ret)):
    p_treynor.append((p_ret[x] - (rf/12)) / p_beta[x])
  portfolio = {'Returns':p_ret, 'Volatility':p_var, 'Beta' : p_beta, 'Sharpe_Ratio' : p_sharpe, 'Treynor_Ratio':p_treynor}
  for x in range(len(capm_data)):
      temp = []
      for y in p_weights:
        temp.append(y[x])
      portfolio[list(capm_data.keys())[x]] = temp
  return pd.DataFrame(portfolio)

capm_min = 0.0001
capm_max = 1
b = (capm_min, capm_max)
capm_bounds = ()
for i in range(capm_num_assets):
  capm_bounds += (b,)

def capm_generate_random_guess():
  x = np.random.random(capm_num_assets)
  x = x/np.sum(x)
  while x.min() < capm_min:
    x = np.random.random(capm_num_assets)
    x = x/np.sum(x)
  return x
capm_x = np.random.random(capm_num_assets)
capm_x = capm_x/np.sum(capm_x)
while capm_x.min() < capm_min:
  capm_x = np.random.random(capm_num_assets)
  capm_x = capm_x/np.sum(capm_x)

method = 'SLSQP'
niter = 15

# capm_mvp = minimize (capm_objective_min_var,capm_x,method='SLSQP',bounds=capm_bounds,constraints=[capm_con1])
capm_mvp = basinhopping (capm_objective_min_var,capm_x,minimizer_kwargs=dict(method=method,bounds=capm_bounds,constraints=[capm_con1]), niter=niter)
capm_mvp_sd = capm_calc_sd(capm_mvp)
capm_mvp_ret = np.dot(capm_mvp['x'], capm_exp_ret_arr)
capm_mvp_beta = np.dot(capm_mvp['x'], capm_beta_arr)
capm_mvp_sharpe = (capm_mvp_ret - (rf/12)) / capm_mvp_sd

# capm_maxvp = minimize (capm_objective_max_var,capm_x,method='SLSQP',bounds=capm_bounds,constraints=[capm_con1])
capm_maxvp = basinhopping (capm_objective_max_var,capm_x,minimizer_kwargs=dict(method=method,bounds=capm_bounds,constraints=[capm_con1]), niter=niter)
capm_maxvp_sd = capm_calc_sd(capm_maxvp)
capm_maxvp_ret = np.dot(capm_maxvp['x'], capm_exp_ret_arr)
capm_maxvp_beta = np.dot(capm_maxvp['x'], capm_beta_arr)
capm_maxvp_sharpe = (capm_maxvp_ret - (rf/12)) / capm_maxvp_sd

capm_diff = capm_maxvp_sd - capm_mvp_sd
capm_step = capm_diff / no_of_steps

capm_p_ret_best = [] # Define an empty array for portfolio returns
capm_p_var_best = [] # Define an empty array for portfolio volatility
capm_p_weights_best = [] # Define an empty array for asset weights
capm_p_beta_best = []
capm_p_sharpe_best = []
capm_p_ret_ef = [] # Define an empty array for portfolio returns
capm_p_var_ef = [] # Define an empty array for portfolio volatility
capm_p_weights_ef = [] # Define an empty array for asset weights
capm_p_beta_ef = []
capm_p_sharpe_ef = []
capm_p_ret_wp = [] # Define an empty array for portfolio returns
capm_p_var_wp = [] # Define an empty array for portfolio volatility
capm_p_weights_wp = [] # Define an empty array for asset weights
capm_p_beta_wp = []
capm_p_sharpe_wp = []

capm_p_ret_best.append(capm_mvp_ret)
capm_p_var_best.append(capm_mvp_sd)
capm_p_weights_best.append(capm_mvp['x'])
capm_p_beta_best.append(capm_mvp_beta)
capm_p_sharpe_best.append(capm_mvp_sharpe)

capm_target_sd = capm_mvp_sd
capm_count_ef = 0
capm_count_wp = 0
capm_ef_x = capm_mvp['x']
capm_wp_x = capm_mvp['x']
for i in tqdm(range(no_of_steps-1)):
  capm_target_sd += capm_step
  # capm_temp = minimize(capm_objective_efficiency_frontier,capm_x,method='SLSQP',bounds=capm_bounds,constraints=[capm_con1, capm_con2])
  capm_temp = basinhopping (capm_objective_efficiency_frontier,capm_ef_x,minimizer_kwargs=dict(method=method,bounds=capm_bounds,constraints=[capm_con1, capm_con2]), niter=niter)
  while capm_temp['success'] != True:
    capm_count_ef += 1
    print("error while optimising count ef = ", capm_count_ef)
    print(capm_temp)
    # capm_temp = minimize(capm_objective_efficiency_frontier,capm_generate_random_guess(),method='SLSQP',bounds=capm_bounds,constraints=[capm_con1, capm_con2])
    capm_temp = basinhopping (capm_objective_efficiency_frontier,capm_ef_x,minimizer_kwargs=dict(method=method,bounds=capm_bounds,constraints=[capm_con1, capm_con2]), niter=niter)

  capm_p_ret_ef.append(np.dot(capm_temp['x'], capm_exp_ret_arr))
  capm_p_var_ef.append(capm_target_sd)
  capm_p_weights_ef.append(capm_temp['x'])
  capm_p_beta_ef.append(np.dot(capm_temp['x'], capm_beta_arr))
  capm_p_sharpe_ef.append((np.dot(capm_temp['x'], capm_exp_ret_arr) - (rf/12)) / capm_target_sd)
  capm_ef_x = capm_temp['x']

  # capm_temp1 = minimize(capm_objective_worst_portfolios,capm_wp_x,method='SLSQP',bounds=capm_bounds,constraints=[capm_con1, capm_con2])
  capm_temp1 = basinhopping (capm_objective_worst_portfolios,capm_wp_x,minimizer_kwargs=dict(method=method,bounds=capm_bounds,constraints=[capm_con1, capm_con2]), niter=niter)
  while capm_temp1['success'] != True:
    capm_count_wp += 1
    print("error while optimising count wp = ", capm_count_wp)
    print(capm_temp1)
    # capm_temp1 = minimize(capm_objective_worst_portfolios,capm_generate_random_guess(),method='SLSQP',bounds=capm_bounds,constraints=[capm_con1, capm_con2])
    capm_temp1 = basinhopping (capm_objective_worst_portfolios,capm_wp_x,minimizer_kwargs=dict(method=method,bounds=capm_bounds,constraints=[capm_con1, capm_con2]), niter=niter)
  capm_p_ret_wp.append(np.dot(capm_temp1['x'], capm_exp_ret_arr))
  capm_p_var_wp.append(capm_target_sd)
  capm_p_weights_wp.append(capm_temp1['x'])
  capm_p_beta_wp.append(np.dot(capm_temp1['x'], capm_beta_arr))
  capm_p_sharpe_wp.append((np.dot(capm_temp1['x'], capm_exp_ret_arr) - (rf/12)) / capm_target_sd)
  capm_wp_x = capm_temp1['x']

  if capm_temp['success'] != True:
    print("error while optimising")
    break
  if capm_temp1['success'] != True:
    print("error while optimising")
    break

capm_p_ret_best.append(capm_maxvp_ret)
capm_p_var_best.append(capm_maxvp_sd)
capm_p_weights_best.append(capm_maxvp['x'])
capm_p_beta_best.append(capm_maxvp_beta)
capm_p_sharpe_best.append(capm_maxvp_sharpe)

# capm_max_sharpe_port = minimize(capm_objective_max_sharpe,capm_generate_random_guess(),method='SLSQP',bounds=capm_bounds,constraints=[capm_con1])
capm_max_sharpe_port = basinhopping(capm_objective_max_sharpe,capm_generate_random_guess(),minimizer_kwargs=dict(method='SLSQP',bounds=capm_bounds,constraints=[capm_con1]), niter=niter)
capm_max_sharpe_port_sd = capm_calc_sd(capm_max_sharpe_port)
capm_max_sharpe_port_ret = np.dot(capm_max_sharpe_port['x'], capm_exp_ret_arr)
capm_max_sharpe_port_beta = np.dot(capm_max_sharpe_port['x'], capm_beta_arr)
capm_max_sharpe_port_sharpe = (capm_max_sharpe_port_ret - (rf/12)) / capm_max_sharpe_port_sd

capm_p_ret_best.append(capm_max_sharpe_port_ret)
capm_p_var_best.append(capm_max_sharpe_port_sd)
capm_p_weights_best.append(capm_max_sharpe_port['x'])
capm_p_beta_best.append(capm_max_sharpe_port_beta)
capm_p_sharpe_best.append(capm_max_sharpe_port_sharpe)

capm_equal_weights = np.array([1/capm_num_assets] * capm_num_assets)
capm_equal_weights_ret = np.dot(capm_equal_weights, capm_exp_ret_arr)
capm_equal_weights_var = 0
for z in range(len(capm_equal_weights)):
  for y in range(len(capm_equal_weights)):
    capm_equal_weights_var += capm_equal_weights[z] * capm_equal_weights[y] * capm_cov_matrix.iloc[z,y]
capm_equal_weights_sd = np.sqrt(capm_equal_weights_var)
capm_equal_weights_beta = np.dot(capm_equal_weights, capm_beta_arr)
capm_equal_weights_sharpe = (capm_equal_weights_ret - (rf/12)) / capm_equal_weights_sd

capm_p_ret_best.append(capm_equal_weights_ret)
capm_p_var_best.append(capm_equal_weights_sd)
capm_p_weights_best.append(capm_equal_weights)
capm_p_beta_best.append(capm_equal_weights_beta)
capm_p_sharpe_best.append(capm_equal_weights_sharpe)

capm_portfolios_best_df = capm_generate_portfolio_df(capm_p_ret_best, capm_p_var_best, capm_p_beta_best, capm_p_sharpe_best, capm_p_weights_best)

capm_portfolios_ef_df = capm_generate_portfolio_df(capm_p_ret_ef, capm_p_var_ef, capm_p_beta_ef, capm_p_sharpe_ef, capm_p_weights_ef)

capm_portfolios_wp_df = capm_generate_portfolio_df(capm_p_ret_wp, capm_p_var_wp, capm_p_beta_wp, capm_p_sharpe_wp, capm_p_weights_wp)

# fig1 = px.scatter(x=portfolios_ef_df['Volatility'].to_list(), y=portfolios_ef_df['Returns'].to_list())
# fig1.show()

capm_plot_best_df = capm_portfolios_best_df.copy(deep=True)
capm_plot_best_df["Classifier"]=["best"]*(len(capm_portfolios_best_df))

capm_plot_ef_df = capm_portfolios_ef_df.copy(deep=True)
capm_plot_ef_df["Classifier"]=["efficient_frontier"]*(len(capm_portfolios_ef_df))

# plot_df = portfolios_df.copy(deep=True)
# plot_df["Classifier"]=["feasible_set"]*(len(portfolios_df))

capm_plot_wp_df = capm_portfolios_wp_df.copy(deep=True)
capm_plot_wp_df["Classifier"]=["worst_portfolios"]*(len(capm_portfolios_wp_df))

capm_plot_final = pd.concat([capm_plot_best_df, capm_plot_ef_df,
                            #  plot_df,
                             capm_plot_wp_df])

capm_fig2 = px.scatter(capm_plot_final,x=capm_plot_final['Volatility'], y=capm_plot_final['Returns'], color="Classifier", color_discrete_map={"efficient_frontier": 'blue', "feasible_set": 'black', "worst_portfolios": 'red', "best":"green"}, hover_data=capm_plot_final, title="CAPM Underpriced Securities Markowitz Frontier")
# fig2.show()

capm_slope = (np.dot(capm_max_sharpe_port['x'], capm_exp_ret_arr) - (rf/12)) / (capm_max_sharpe_port_sd - 0)
capm_y = (rf/12) + (capm_slope * 0.1)

temp = {"Volatility":[0,0.1], "Returns":[(rf/12), capm_y]}
temp_df = pd.DataFrame(temp)

capm_fig2.add_trace(px.line(temp_df, x=temp_df['Volatility'], y=temp_df['Returns']).data[0])
capm_fig2.show()

# Superimpose graph ka code
# fig2.add_trace(px.line(temp_df, x=temp_df['Volatility'], y=temp_df['Returns']).data[0])

100%|██████████| 74/74 [38:57<00:00, 31.59s/it] 


In [100]:
# SML Plot
classifier, ticker,capm_return, beta, mean_return = [],[],[],[],[]
for x in derived_data:
  capm_return.append(derived_data[x]["expected_return_capm_monthly"])
  beta.append(derived_data[x]["beta"])
  mean_return.append(derived_data[x]["mean"])
  ticker.append(x)
  if derived_data[x]["mean"] > derived_data[x]["expected_return_capm_monthly"]:
    classifier.append("Underpriced")
  else:
    classifier.append("Overpriced")


data_return = {'ticker':ticker,
        'beta': beta,
        'capm_return': capm_return}
sml_df = pd.DataFrame(data_return)

# Linear equation parameters
intercept = rf / 12
slope = rm - (rf/12)

data2 = {'ticker':ticker, 'beta': beta, 'mean_return': mean_return, "classifier": classifier}
avg_ret_df = pd.DataFrame(data2)

# PLOTTING
capm_fig3 = px.scatter(avg_ret_df,x=avg_ret_df['beta'], y=avg_ret_df['mean_return'], hover_data=avg_ret_df, text=avg_ret_df['ticker'], color="classifier",  title = "SML and Mean Monthly Return")
capm_fig3.update_traces(
    textposition = "top right"
)
capm_fig3.add_trace(px.line(sml_df, x=sml_df['beta'], y=sml_df['capm_return']).data[0])
capm_fig3.update_layout(
    yaxis_title="Return"
    # title = "SML and Mean Monthly Return",
)
capm_fig3.show()


# Download Data

In [101]:
# Raw data downloaded from yf
for x in data:
  data[x].to_csv("./data/data_"+x+".csv")
niftyPSE.to_csv("./data/niftyPSE.csv")

pd.DataFrame(derived_data).to_csv("./data/derived_data.csv") # basic statistics
corr_matrix.to_csv("./data/corr_matrix.csv")
cov_matrix.to_csv("./data/cov_matrix.csv")

# MARKOWITZ
portfolios_df.to_csv("./data/portfolios_df.csv") # random generated portfolios (black group)
portfolios_best_df.to_csv("./data/portfolios_best_df.csv") # 4 Unique portfolios - MVP, Max VP, Dominant by Sharpe, Equi weighter
portfolios_ef_df.to_csv("./data/portfolios_ef_df.csv") # Efficient frontier portfolios
portfolios_wp_df.to_csv("./data/portfolios_wp_df.csv") # Worst frontier portfolios
plot_final.to_csv("./data/plot_final.csv") # All combined (Return, volatility, classifier) with wts

# SHARPE'S OPTIMAL
sharpe_df_original.to_csv("./data/sharpe_df_original.csv") # Securities C* Table
sharpe_df.to_csv("./data/sharpe_df.csv") # selected securities till C* + Weights of optimal portfolio
sharpe_portfolio_df.to_csv("./data/sharpe_portfolio_df.csv") # Optimal portfolio

# CAPM
capm_cov_matrix.to_csv("./data/capm_cov_matrix.csv") # capm underpriced secs' cov matrix
capm_portfolios_best_df.to_csv("./data/capm_portfolios_best_df.csv") # 4 unique portfolios
capm_portfolios_ef_df.to_csv("./data/capm_portfolios_ef_df.csv") # efficient frontier
capm_portfolios_wp_df.to_csv("./data/capm_portfolios_wp_df.csv") # worst portfolios
capm_plot_final.to_csv("./data/capm_plot_final.csv") # All combined

# !zip -r /content/data.zip /content/data

# files.download("/content/data.zip")

# Arbitrage Pricing Theory

## Getting Market Data

In [ ]:
abt_start_date = datetime(2020,5,1)
# abt_start_date = datetime(2018,5,1)
abt_end_date = datetime(2023,7,1)
# abt_no_of_years = 5
abt_no_of_years = 3

abt_niftyPSE = yf.Ticker(market_ticker).history(start=abt_start_date, end=abt_end_date, interval='3mo')

temp = [0]
for y in range(1,len(abt_niftyPSE["Open"])): #start from 1 because 1st entry mai return nahi aayenge
  temp.append((((abt_niftyPSE["Open"][y] + abt_niftyPSE["Dividends"][y])) / abt_niftyPSE["Open"][y-1]) - 1)

abt_niftyPSE["Return"] = temp

# abt_rm = ((abt_niftyPSE["Open"][-1] / abt_niftyPSE["Open"][1]) ** (1 / abt_no_of_years)) - 1
abt_rm = abt_niftyPSE["Return"].iloc[1:].mean()

abt_data = {}

for x in tickers:
  if len(yf.Ticker(x + ".NS").history(start=abt_start_date, end=abt_end_date, interval='3mo')) == ((abt_no_of_years*4) + 1):
    abt_data[x] = yf.Ticker(x + ".NS").history(start=abt_start_date, end=abt_end_date, interval='3mo')
  else:
    print(x, len(yf.Ticker(x + ".NS").history(start=abt_start_date, end=abt_end_date, interval='3mo')))

abt_data

LICI 5


{'BEL':                                 Open        High        Low       Close  \
 Date                                                                      
 2020-04-01 00:00:00+05:30  22.146232   28.934467  17.738487   27.229729   
 2020-07-01 00:00:00+05:30  27.229727   36.383094  26.891851   29.410563   
 2020-10-01 00:00:00+05:30  29.955908   38.343563  26.874729   37.332066   
 2021-01-01 00:00:00+05:30  37.658853   48.225120  36.056017   38.934898   
 2021-04-01 00:00:00+05:30  40.048751   56.862867  38.300588   56.576805   
 2021-07-01 00:00:00+05:30  56.513236   67.526642  51.602495   64.507095   
 2021-10-01 00:00:00+05:30  63.960041   72.914449  58.667351   67.142052   
 2022-01-01 00:00:00+05:30  67.078091   72.434747  59.802637   67.413879   
 2022-04-01 00:00:00+05:30  68.468114   84.226006  68.240918   75.981918   
 2022-07-01 00:00:00+05:30  75.624895  111.635982  72.752450   98.296135   
 2022-10-01 00:00:00+05:30  99.322491  109.847053  92.665098   97.805000   
 2023

## Finding Lamdas for Macroeconomic Factors

In [ ]:
sbi_psu = yf.Ticker("0P0000XVLF.BO").history(start=abt_start_date, end=abt_end_date, interval='3mo') # SBI PSU Dir Gir https://finance.yahoo.com/quote/0P0000XVLF.BO?.tsrc=fin-srch
invesco_psu = yf.Ticker("0P0000XVHQ.BO").history(start=abt_start_date, end=abt_end_date, interval='3mo') # Invesco PSU Dir Gir https://finance.yahoo.com/quote/0P0000XVHQ.BO?.tsrc=fin-srch
# dsp = yf.Ticker("0P0000XW2E.BO").history(start=abt_start_date, end=abt_end_date, interval='3mo') # NIFTY Oil https://finance.yahoo.com/quote/NIFTY_OIL_AND_GAS.NS?.tsrc=fin-srch
# nifty50 = yf.Ticker("^NSEI").history(start=abt_start_date, end=abt_end_date, interval='3mo') # NIFTY 50

temp = [0]
for y in range(1,len(sbi_psu["Open"])): #start from 1 because 1st entry mai return nahi aayenge
  temp.append((((sbi_psu["Open"][y] + sbi_psu["Dividends"][y])) / sbi_psu["Open"][y-1]) - 1)

sbi_psu["Return"] = temp
# sbi_psu_ret = ((sbi_psu["Open"][-1] / sbi_psu["Open"][1]) ** (1 / abt_no_of_years)) - 1
sbi_psu_ret = sbi_psu["Return"].iloc[1:].mean()

temp = [0]
for y in range(1,len(invesco_psu["Open"])): #start from 1 because 1st entry mai return nahi aayenge
  temp.append((((invesco_psu["Open"][y] + invesco_psu["Dividends"][y])) / invesco_psu["Open"][y-1]) - 1)

invesco_psu["Return"] = temp
# invesco_psu_ret = ((invesco_psu["Open"][-1] / invesco_psu["Open"][1]) ** (1 / abt_no_of_years)) - 1
invesco_psu_ret = invesco_psu["Return"].iloc[1:].mean()


invesco_psu

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,Return
Date,,,,,,,,,
2020-04-01 00:00:00+05:30,18.059999,0.0,0.0,19.930000,0,0.0,0.0,0.0,0.000000
2020-07-01 00:00:00+05:30,19.969999,0.0,0.0,19.170000,0,0.0,0.0,0.0,0.105759
2020-10-01 00:00:00+05:30,19.180000,0.0,0.0,21.830000,0,0.0,0.0,0.0,-0.039559
2021-01-01 00:00:00+05:30,21.969999,0.0,0.0,24.090000,0,0.0,0.0,0.0,0.145464
2021-04-01 00:00:00+05:30,24.520000,0.0,0.0,27.920000,0,0.0,0.0,0.0,0.116067
2021-07-01 00:00:00+05:30,27.959999,0.0,0.0,30.559999,0,0.0,0.0,0.0,0.140294
2021-10-01 00:00:00+05:30,30.500000,0.0,0.0,29.020000,0,0.0,0.0,0.0,0.090844
2022-01-01 00:00:00+05:30,29.469999,0.0,0.0,29.980000,0,0.0,0.0,0.0,-0.033771
2022-04-01 00:00:00+05:30,30.830000,0.0,0.0,27.510000,0,0.0,0.0,0.0,0.046149


In [ ]:
# nifty_auto = yf.Ticker("^CNXAUTO").history(start=abt_start_date, end=abt_end_date, interval='3mo')

# nifty_auto

In [ ]:
macroeconomic_factors = pd.read_csv("macroeconomic_factors.csv").iloc[8:,:].reset_index(drop=True)

temp = [0]
for y in range(1,len(macroeconomic_factors["gdp"])): #start from 1 because 1st entry mai return nahi aayenge
  temp.append((((macroeconomic_factors["gdp"][y])) / macroeconomic_factors["gdp"][y-1]) - 1)

macroeconomic_factors["gdp_qoq"] = temp

temp = [0]
for y in range(1,len(macroeconomic_factors["gold"])): #start from 1 because 1st entry mai return nahi aayenge
  temp.append((((macroeconomic_factors["gold"][y])) / macroeconomic_factors["gold"][y-1]) - 1)

macroeconomic_factors["gold_qoq"] = temp

temp = [0]
for y in range(1,len(macroeconomic_factors["oil"])): #start from 1 because 1st entry mai return nahi aayenge
  temp.append((((macroeconomic_factors["oil"][y])) / macroeconomic_factors["oil"][y-1]) - 1)

macroeconomic_factors["oil_qoq"] = temp

macroeconomic_factors

,date,gdp,gold,oil,gdp_qoq,gold_qoq,oil_qoq
0,"Apr 1, 2020",3821081,15278.89,1959.30,0.000000,0.000000,0.000000
1,"Jul 1, 2020",2724023,18091.52,3613.34,-0.287107,0.184086,0.844199
2,"Oct 1, 2020",3346731,17146.63,3061.47,0.228599,-0.052228,-0.152731
3,"Jan 1, 2021",3660581,16592.93,4340.74,0.093778,-0.032292,0.417861
4,"Apr 1, 2021",3955784,15573.65,5287.75,0.080644,-0.061429,0.218168
5,"Jul 1, 2021",3311050,15771.27,6185.03,-0.162985,0.012689,0.169690
6,"Oct 1, 2021",3651569,15621.49,7054.84,0.102843,-0.009497,0.140631
7,"Jan 1, 2022",3850772,15475.59,7529.41,0.054553,-0.009340,0.067269
8,"Apr 1, 2022",4112360,16663.87,9682.78,0.067931,0.076784,0.285995
9,"Jul 1, 2022",3744285,16553.11,9889.79,-0.089505,-0.006647,0.021379


In [ ]:
var_gdp = pd.DataFrame({'sbi_psu':sbi_psu["Return"].iloc[1:].tolist(), "gdp":macroeconomic_factors["gdp_qoq"].iloc[1:].tolist()}).cov().iloc[1,1]
var_gold = pd.DataFrame({'sbi_psu':sbi_psu["Return"].iloc[1:].tolist(), "gold":macroeconomic_factors["gold_qoq"].iloc[1:].tolist()}).cov().iloc[1,1]
var_oil = pd.DataFrame({'sbi_psu':sbi_psu["Return"].iloc[1:].tolist(), "oil":macroeconomic_factors["oil_qoq"].iloc[1:].tolist()}).cov().iloc[1,1]
var_niftypse = pd.DataFrame({'sbi_psu':sbi_psu["Return"].iloc[1:].tolist(), "nifty":abt_niftyPSE["Return"].iloc[1:].tolist()}).cov().iloc[1,1]

oil_cov_sbi = pd.DataFrame({'sbi_psu':sbi_psu["Return"].iloc[1:].tolist(), "oil":macroeconomic_factors["oil_qoq"].iloc[1:].tolist()}).cov().iloc[0,1]
niftypse_cov_sbi = pd.DataFrame({'sbi_psu':sbi_psu["Return"].iloc[1:].tolist(), "nifty":abt_niftyPSE["Return"].iloc[1:].tolist()}).cov().iloc[0,1]
gdp_cov_sbi = pd.DataFrame({'sbi_psu':sbi_psu["Return"].iloc[1:].tolist(), "gdp":macroeconomic_factors["gdp_qoq"].iloc[1:].tolist()}).cov().iloc[0,1]
gold_cov_sbi = pd.DataFrame({'sbi_psu':sbi_psu["Return"].iloc[1:].tolist(), "gold":macroeconomic_factors["gold_qoq"].iloc[1:].tolist()}).cov().iloc[0,1]
var_sbi = pd.DataFrame({'sbi_psu':sbi_psu["Return"].iloc[1:].tolist(), "gdp":macroeconomic_factors["gdp_qoq"].iloc[1:].tolist()}).cov().iloc[0,0]

sbi_beta_gdp = gdp_cov_sbi / var_gdp
sbi_beta_gold = gold_cov_sbi / var_gold
sbi_beta_oil = oil_cov_sbi / var_oil
sbi_beta_niftypse = niftypse_cov_sbi / var_niftypse

oil_cov_invesco = pd.DataFrame({'invesco_psu':invesco_psu["Return"].iloc[1:].tolist(), "oil":macroeconomic_factors["oil_qoq"].iloc[1:].tolist()}).cov().iloc[0,1]
niftypse_cov_invesco = pd.DataFrame({'invesco_psu':invesco_psu["Return"].iloc[1:].tolist(), "nifty":abt_niftyPSE["Return"].iloc[1:].tolist()}).cov().iloc[0,1]
gdp_cov_invesco = pd.DataFrame({'invesco_psu':invesco_psu["Return"].iloc[1:].tolist(), "gdp":macroeconomic_factors["gdp_qoq"].iloc[1:].tolist()}).cov().iloc[0,1]
gold_cov_invesco = pd.DataFrame({'invesco_psu':invesco_psu["Return"].iloc[1:].tolist(), "gold":macroeconomic_factors["gold_qoq"].iloc[1:].tolist()}).cov().iloc[0,1]
var_invesco = pd.DataFrame({'invesco_psu':invesco_psu["Return"].iloc[1:].tolist(), "gdp":macroeconomic_factors["gdp_qoq"].iloc[1:].tolist()}).cov().iloc[0,0]

invesco_beta_niftypse = niftypse_cov_invesco / var_niftypse
invesco_beta_gdp = gdp_cov_invesco / var_gdp
invesco_beta_oil = oil_cov_invesco / var_oil
invesco_beta_gold = gold_cov_invesco / var_gold

lamda_gdp, lamda_gold = symbols('lambda_gdp, lambda_gold')

eq1 = Eq((rf + sbi_beta_niftypse * (abt_rm - rf) + sbi_beta_gdp * lamda_gdp + sbi_beta_gold * lamda_gold), sbi_psu_ret)
eq2 = Eq((rf + invesco_beta_niftypse * (abt_rm - rf) + invesco_beta_gdp * lamda_gdp + invesco_beta_gold * lamda_gold), invesco_psu_ret)

solved_eqs = solve((eq1, eq2), (lamda_gdp, lamda_gold))

temp = []
for x in solved_eqs.values():
  temp.append(x)

lambda_gdp = temp[0]
lambda_gold = temp[1]

lamda_oil = symbols('lambda_oil')

eq1_1 = Eq((abt_rf + invesco_beta_niftypse * (abt_rm - abt_rf) + invesco_beta_oil * lamda_oil), invesco_psu_ret)
# eq2 = Eq((rf + invesco_beta_niftypse * (abt_rm - rf) + invesco_beta_gdp * lamda_gdp + invesco_beta_gold * lamda_gold), invesco_psu_ret)

solved_eqs1 = solve((eq1_1), (lamda_oil))

solved_eqs1

[0.0864756472799001]

## Getting Market and Share Data

In [ ]:
# Calculating month on month return, x-xbar and (x-xbar)^2

for x in abt_data:
  temp = [0]
  count = 0
  for y in range(1,len(abt_data[x]["Open"])): #start from 1 because 1st entry mai return nahi aayenge
    temp.append((((abt_data[x]["Open"][y] + abt_data[x]["Dividends"][y])) / abt_data[x]["Open"][y-1]) - 1)

  abt_data[x]["Return"] = temp

abt_derived_data = {}

for x in abt_data:
  abt_cov_wrt_index = pd.DataFrame({x:abt_data[x]["Return"].iloc[1:].tolist(), "Index":abt_niftyPSE["Return"].iloc[1:].tolist()}).cov().iloc[0,1]
  abt_var_index = pd.DataFrame({x:abt_data[x]["Return"].iloc[1:].tolist(), "Index":abt_niftyPSE["Return"].iloc[1:].tolist()}).cov().iloc[1,1]
  beta_index = abt_cov_wrt_index / abt_var_index

  abt_cov_wrt_gdp = pd.DataFrame({x:abt_data[x]["Return"].iloc[1:].tolist(), "gdp":macroeconomic_factors["gdp_qoq"].iloc[1:].tolist()}).cov().iloc[0,1]
  abt_var_gdp = pd.DataFrame({x:abt_data[x]["Return"].iloc[1:].tolist(), "gdp":macroeconomic_factors["gdp_qoq"].iloc[1:].tolist()}).cov().iloc[1,1]
  beta_gdp = abt_cov_wrt_gdp / abt_var_gdp

  abt_cov_wrt_oil = pd.DataFrame({x:abt_data[x]["Return"].iloc[1:].tolist(), "oil":macroeconomic_factors["oil_qoq"].iloc[1:].tolist()}).cov().iloc[0,1]
  abt_var_oil = pd.DataFrame({x:abt_data[x]["Return"].iloc[1:].tolist(), "oil":macroeconomic_factors["oil_qoq"].iloc[1:].tolist()}).cov().iloc[1,1]
  beta_oil = abt_cov_wrt_oil / abt_var_oil

  abt_cov_wrt_gold = pd.DataFrame({x:abt_data[x]["Return"].iloc[1:].tolist(), "gold":macroeconomic_factors["gold_qoq"].iloc[1:].tolist()}).cov().iloc[0,1]
  abt_var_gold = pd.DataFrame({x:abt_data[x]["Return"].iloc[1:].tolist(), "gold":macroeconomic_factors["gold_qoq"].iloc[1:].tolist()}).cov().iloc[1,1]
  beta_gold = abt_cov_wrt_gold / abt_var_gold

  # expected_return_abt = (rf + (beta_index * (abt_rm - rf)) + (beta_gdp * lambda_gdp) + (beta_gold * lambda_gold))
  # expected_return_abt = (rf + (beta_index * (abt_rm - rf)) (beta_gold * solved_eqs[0]))
  expected_return_abt = (rf + (beta_index * (abt_rm - rf)) + (beta_oil * solved_eqs1[0]))

  abt_derived_data[x] = {
      "mean" : abt_data[x]["Return"].iloc[1:].mean(),
      "beta_index" : beta_index,
      "beta_gdp" : beta_gdp,
      "beta_gold" : beta_gold,
      "beta_oil" : beta_oil,
      "expected_return_abt" : expected_return_abt,
      "expected_return_abt_quaterly" : (expected_return_abt / 4),
  }

abt_derived_data

{'BEL': {'mean': 0.14713852136878677,
  'beta_index': 0.6366192041525424,
  'beta_gdp': -0.49250550697952206,
  'beta_gold': -0.16820042193058324,
  'beta_oil': 0.17229099584040636,
  'expected_return_abt': 0.0794212275615376,
  'expected_return_abt_quaterly': 0.0198553068903844},
 'BHEL': {'mean': 0.1303484610659674,
  'beta_index': 1.3175166004260808,
  'beta_gdp': -1.216413383926021,
  'beta_gold': 1.381790060164075,
  'beta_oil': 0.5077933409553024,
  'expected_return_abt': 0.0984040202941771,
  'expected_return_abt_quaterly': 0.0246010050735443},
 'BPCL': {'mean': 0.027915501773933694,
  'beta_index': 0.8153861657187486,
  'beta_gdp': -0.07732133538679517,
  'beta_gold': -0.10556416702399961,
  'beta_oil': 0.08637843856745046,
  'expected_return_abt': 0.0693585487929358,
  'expected_return_abt_quaterly': 0.0173396371982340},
 'COALINDIA': {'mean': 0.10540655587310632,
  'beta_index': 1.1320853150988242,
  'beta_gdp': 0.294786604729134,
  'beta_gold': -0.3283726265009392,
  'beta_o

In [ ]:
abt_data_final = {}
for x in abt_derived_data:
    if abt_derived_data[x]['mean'] > abt_derived_data[x]['expected_return_abt_quaterly']:
        abt_data_final[x] = abt_data[x]

abt_exp_ret_arr = []
for x in abt_derived_data:
    if x in list(abt_data_final.keys()):
        abt_exp_ret_arr.append(abt_derived_data[x]['mean'])

abt_num_assets = len(list(abt_data_final.keys()))

abt_cov_data = {}

for x in abt_data_final:
  abt_cov_data[x] = abt_data_final[x]["Return"].iloc[1:].tolist()

abt_cov_df = pd.DataFrame(abt_cov_data)

abt_cov_matrix = abt_cov_df.cov()

abt_cov_matrix

,BEL,BHEL,BPCL,COALINDIA,CONCOR,GAIL,HAL,HINDPETRO,IOC,IRCTC,NHPC,NMDC,NTPC,ONGC,OIL,PFC,POWERGRID,RECLTD,SAIL
BEL,0.019104,0.020602,0.005707,0.005919,0.008235,0.003704,0.002891,0.013479,0.005903,0.008456,0.003109,0.010878,0.004128,0.002382,-0.002710,0.002431,0.001566,0.009385,0.033026
BHEL,0.020602,0.071048,0.017170,-0.000476,0.034734,0.012765,0.015890,0.022842,0.010268,0.006380,0.005402,0.027000,0.005561,0.005711,-0.008206,0.008395,0.005725,0.022065,0.032274
BPCL,0.005707,0.017170,0.010461,0.004035,0.013575,0.011641,-0.001318,0.012043,0.010053,0.006147,0.002753,0.017939,0.004588,0.009242,0.009557,0.010726,0.003979,0.009171,0.026885
COALINDIA,0.005919,-0.000476,0.004035,0.029466,0.007471,0.017281,0.006723,0.009228,0.016601,0.021837,0.005142,0.014380,0.014148,0.018503,0.025259,0.016911,0.002222,0.012850,0.022355
CONCOR,0.008235,0.034734,0.013575,0.007471,0.036661,0.012532,0.010588,0.012621,0.009786,0.014019,0.003854,0.025760,0.007317,0.007416,0.001498,0.003547,0.005554,0.006369,0.018025
GAIL,0.003704,0.012765,0.011641,0.017281,0.012532,0.033503,0.001506,0.013635,0.024399,0.001768,0.001623,0.031390,0.012130,0.025290,0.024875,0.022207,0.008143,0.019491,0.048035
HAL,0.002891,0.015890,-0.001318,0.006723,0.010588,0.001506,0.023650,-0.003078,-0.001577,0.012744,-0.001506,-0.010107,0.005796,-0.002480,0.002352,-0.008066,-0.000306,-0.000637,-0.022442
HINDPETRO,0.013479,0.022842,0.012043,0.009228,0.012621,0.013635,-0.003078,0.020100,0.015499,0.014162,0.004576,0.021835,0.007087,0.014567,0.014210,0.014232,0.005211,0.014201,0.040984
IOC,0.005903,0.010268,0.010053,0.016601,0.009786,0.024399,-0.001577,0.015499,0.024506,0.012621,0.004491,0.021543,0.011104,0.024586,0.028470,0.020528,0.007307,0.018231,0.041963
IRCTC,0.008456,0.006380,0.006147,0.021837,0.014019,0.001768,0.012744,0.014162,0.012621,0.075738,0.007186,-0.002902,0.012797,0.016315,0.033205,0.002845,0.007014,-0.001054,0.001078
